In [1]:
# ! pip install transformers
import torch
import numpy as np
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW

from torch import nn
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df[df['content']==' ']

In [3]:
torch.cuda.empty_cache()

In [4]:
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('BERT_data.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nos.chdir('/content/drive/My Drive/')\n\n\ndf=pd.read_csv('BERT_data.csv')\ndf=df[~(df['content']=='nan')]\ndf['content']=df['content'].astype(str)\ndf['subject']=df['subject'].astype(str)\n"

In [5]:

df=pd.read_csv('BERT_data.csv')
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)


In [6]:
df['cat_1_level_2']=df['cat_1_level_2']-1
df['cat_1_level_2'].unique()

array([0, 2, 3, 4, 1, 6, 5, 7], dtype=int64)

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['cat_1_level_2']
        self.text=[tokenizer(text,
                               padding='max_length',truncation=True,
                                return_tensors="pt") for text in df['content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [8]:

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.60*len(df)), int(.90*len(df))])


In [9]:
'''
from sklearn.utils import resample
for i in [1,2,3,4,5,6]:
    spam_upsample = resample(df_train[df_train['cat_1_level_2']==i],
             replace=True,
             n_samples=len(df_train[df_train['cat_1_level_2']==0]) - len(df_train[df_train['cat_1_level_2']==i]),
             random_state=42)
    df_train=pd.concat([df_train,spam_upsample])
df_train.shape
'''

"\nfrom sklearn.utils import resample\nfor i in [1,2,3,4,5,6]:\n    spam_upsample = resample(df_train[df_train['cat_1_level_2']==i],\n             replace=True,\n             n_samples=len(df_train[df_train['cat_1_level_2']==0]) - len(df_train[df_train['cat_1_level_2']==i]),\n             random_state=42)\n    df_train=pd.concat([df_train,spam_upsample])\ndf_train.shape\n"

In [10]:
df_train.reset_index(inplace=True)
df_train=df_train.drop('index',axis=1)

df_val.reset_index(inplace=True)
df_val=df_val.drop('index',axis=1)

df_test.reset_index(inplace=True)
df_test=df_test.drop('index',axis=1)

print(len(df_train),len(df_val), len(df_test))

894 447 149


In [11]:
'''
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=8)
'''

'\nmodel_name = "bert-base-uncased"\nmodel = BertForSequenceClassification.from_pretrained(model_name, num_labels=8)\n'

In [12]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.3):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 8)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [13]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=5)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=5)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            n=0
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)


                output = model(input_id, mask)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

EPOCHS = 300
model = BertClassifier()
LR = 0.000001

train(model, df_train, df_val, LR, EPOCHS)

100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [01:12<00:00,  2.47it/s]


Epochs: 1 | Train Loss:  0.370                 | Train Accuracy:  0.434                 | Val Loss:  0.345                 | Val Accuracy:  0.497


100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [01:10<00:00,  2.55it/s]


Epochs: 2 | Train Loss:  0.328                 | Train Accuracy:  0.504                 | Val Loss:  0.311                 | Val Accuracy:  0.523


100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [01:10<00:00,  2.54it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [01:10<00:00,  2.52it/s]


Epochs: 81 | Train Loss:  0.054                 | Train Accuracy:  0.905                 | Val Loss:  0.286                 | Val Accuracy:  0.622


100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [01:11<00:00,  2.52it/s]


Epochs: 82 | Train Loss:  0.055                 | Train Accuracy:  0.907                 | Val Loss:  0.286                 | Val Accuracy:  0.622


100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [01:11<00:00,  2.51it/s]


In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)

In [ ]:
torch.save(model.state_dict(), 'BERT_model.pth')